In [207]:
import scripts.proj1_helpers as helper
import run as run
import implementations as imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import cross_validation as cv
from preprocessor import Preprocessor
from least_squares import LeastSquares
from model_ensembler import Model_Ensembler
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Import data

In [158]:
y_train, x_train, ids_train = helper.load_csv_data('train.csv')
y_test, x_test, ids_test = helper.load_csv_data('test.csv')

### Test

In [234]:
def preprocess_train(x_tr, deg=1):
    x = x_tr.copy()
    stds = np.std(x, axis=0)
    deleted_cols_ids = np.where(stds == 0)
    x = np.delete(x, deleted_cols_ids, axis=1)
    run.mean_spec(x)
    x = run.standardize(x)
    x = run.polynomial_enhancement(x, deg)
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    return x, deleted_cols_ids

def preprocess_test(x_te, dependency, deg=1): 
    x = x_te.copy()
    stds = np.std(x, axis=0)
    x = np.delete(x, dependency, axis=1)
    run.mean_spec(x)
    x = run.standardize(x)
    x = run.polynomial_enhancement(x, deg)
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    return x

preprocess_train_model_1 = lambda x_tr: preprocess_train(x_tr, deg=1)
preprocess_test_model_1 = lambda x_te, dependency: preprocess_test(x_te, dependency, deg=1)    

def preprocess_train_meta_model(x_tr):
    return run.polynomial_enhancement(x_tr, 1), None
    
def preprocess_test_meta_model(x_te, dependency):
    return run.polynomial_enhancement(x_te, 1)
    

#preprocess_train_meta_model = lambda x_tr: run.polynomial_enhancement(x_tr, 4), None
#preprocess_test_meta_model = lambda x_te: run.polynomial_enhancement(x_te, 4)

In [235]:
preprocessor_1 = Preprocessor(preprocess_train, preprocess_test)
preprocessor_2 = Preprocessor(preprocess_train_model_1, preprocess_test_model_1)
preprocessor_meta = Preprocessor(preprocess_train_meta_model, preprocess_test_meta_model)

In [236]:
least_square_model_1 = LeastSquares(preprocessor_1)
least_square_model_2 = LeastSquares(preprocessor_2)

least_square_meta_model = LeastSquares(preprocessor_meta)

In [237]:
model_ensembler = Model_Ensembler([least_square_model_1, least_square_model_2], least_square_meta_model)

In [238]:
accuracy = cv.cross_validation(y_train, x_train, 5, model_ensembler, seed=1, compute_loss=imp.rmse)
accuracy 

[0.63260000000000005,
 0.65788000000000002,
 0.67259999999999998,
 0.34338000000000002,
 0.34460000000000002]

In [213]:
sum(accuracy) / len(accuracy)

0.59623199999999998

In [56]:
least_square_model.train(y_train, x_train)